In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [11]:
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry import trace
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter

# 1. Set up the provider and exporter (OTLP exporter uses env vars for config)
tracer_provider = TracerProvider()
otlp_exporter = OTLPSpanExporter()
processor = SimpleSpanProcessor(otlp_exporter)

tracer_provider.add_span_processor(processor)

# 2. Register the provider as global
trace.set_tracer_provider(tracer_provider)

Overriding of current TracerProvider is not allowed


In [12]:
# Check SK environment variables are set
import os
assert os.getenv("SEMANTICKERNEL_EXPERIMENTAL_GENAI_ENABLE_OTEL_DIAGNOSTICS_SENSITIVE") == "true"

In [13]:
import os
from openai import AsyncAzureOpenAI
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential, "https://cognitiveservices.azure.com/.default"
)


client = AsyncAzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        azure_ad_token_provider=token_provider,
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    )
service = AzureChatCompletion(
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        async_client=client,
    )

In [14]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.agents.chat_completion.chat_completion_agent import ChatHistoryAgentThread

thread = ChatHistoryAgentThread()

agent = ChatCompletionAgent(
    id="chat-completion-agent",
    name="ChatCompletionAgent",
    service=service,
    instructions="""You are a helpful assistant. You tell jokes""",
)

In [ ]:
async for r in agent.invoke(messages="A joke about developers", thread=thread):
    print(r)

Transient error StatusCode.UNAVAILABLE encountered while exporting traces to localhost:4317, retrying in 1s.
Transient error StatusCode.UNAVAILABLE encountered while exporting traces to localhost:4317, retrying in 2s.
Transient error StatusCode.UNAVAILABLE encountered while exporting traces to localhost:4317, retrying in 4s.


In [ ]:
# finished = memory_exporter.get_finished_spans()
# print(f"Captured {len(finished)} spans: {[span.name for span in finished]}")